### Modelos Pré-Treinados

#### O que são modelos pré-treinados?

Modelos pré-treinados são redes neurais que já foram treinadas em grandes conjuntos de dados, como ImageNet, para resolver tarefas gerais, como classificação de imagens. Esses modelos capturam representações úteis dos dados, como características de baixo nível (bordas, texturas) e de alto nível (formas complexas e objetos inteiros). Por isso, podem ser utilizados como ponto de partida para resolver problemas específicos sem a necessidade de treinar uma rede do zero, economizando tempo e recursos computacionais.

#### Vantagens do Uso de Modelos Pré-Treinados
- **Redução de custos:** O treinamento de grandes redes neurais do zero requer recursos computacionais significativos e tempo.
- **Generalização:** Modelos pré-treinados frequentemente generalizam melhor para novas tarefas devido à riqueza de seus dados de treinamento.
- **Transfer Learning:** Ajustar um modelo pré-treinado para uma tarefa específica é simples. Isso pode ser feito ao congelar camadas inferiores e treinar apenas as camadas superiores para o novo domínio.

#### Como utilizá-los?
Os modelos pré-treinados geralmente vêm acompanhados de pesos otimizados para tarefas específicas e métodos para pré-processar os dados de entrada. Em bibliotecas como o PyTorch, essas funcionalidades estão disponíveis de forma acessível.

#### Hubs de Modelos Pré-Treinados
Existem hubs especializados para encontrar e utilizar modelos pré-treinados. Um dos mais conhecidos é o **torchvision**, que oferece uma variedade de modelos treinados no ImageNet e outras tarefas relacionadas a visão computacional. Para acessar os modelos disponíveis e sua documentação, você pode visitar o [hub oficial do torchvision](https://pytorch.org/vision/main/models.html).

In [ ]:
import torch
import requests
import numpy as np
from PIL import Image
from torchvision import models
import matplotlib.pyplot as plt

In [ ]:
# Importa os pesos pré-treinados do modelo ResNet50 da biblioteca torchvision
weights = models.ResNet50_Weights.IMAGENET1K_V1
# Instancia o modelo ResNet50 com os pesos pré-treinados no ImageNet
model = models.resnet50(weights=weights)

In [ ]:
# Função para carregar uma imagem a partir de uma URL
def load_image_from_url(url):
    response = requests.get(url)  # Faz uma requisição HTTP para obter a imagem
    img = Image.open(requests.get(url, stream=True).raw)  # Abre a imagem diretamente do stream
    return img  # Retorna a imagem carregada

# Função para plotar a imagem carregada
def plot_image(img):
    img = np.array(img)
    plt.imshow(img)
    plt.axis("off")
    plt.show()

# Função para obter as 5 principais previsões do modelo
def get_top5_predictions(model, img, weights=weights):
    # Inferência
    model.eval()
    with torch.no_grad():
        output = model(img)

    # Converte logits em probabilidades
    probabilities = torch.nn.functional.softmax(output[0], dim=0)

    # Obtém as 5 maiores probabilidades e seus índices
    top5_prob, top5_catid = torch.topk(probabilities, 5)

    # Obtém as categorias associadas ao conjunto de pesos
    categories = weights.meta["categories"]
    
    # Cria uma lista das 5 principais categorias com suas respectivas probabilidades
    top5 = [(categories[catid], prob.item()) for catid, prob in zip(top5_catid, top5_prob)]
    return top5

In [ ]:
# Lista de URLs de imagens para teste
urls = [
    "http://images.cocodataset.org/test-stuff2017/000000024309.jpg",
    "http://images.cocodataset.org/test-stuff2017/000000028117.jpg",
    "http://images.cocodataset.org/test-stuff2017/000000006149.jpg",
    "http://images.cocodataset.org/test-stuff2017/000000004954.jpg",
]

# Obtém a transformação recomendada para pré-processar as imagens
# A transformação é configurada automaticamente para o conjunto de pesos utilizados
preprocess = weights.transforms()

# Para cada URL...
for url in urls:
    # Carrega a imagem da URL fornecida
    image = load_image_from_url(url)

    # Mostra a imagem carregada em um gráfico
    plot_image(image)

    # Pré-processa a imagem para que tenha o formato esperado pelo modelo
    image = preprocess(image).unsqueeze(0)

    # Obtém as 5 principais previsões do modelo para a imagem processada
    predictions = get_top5_predictions(model, image)

    for cat, prob in predictions:
        print(f"{cat}: {prob:.2f}")
    print()

## Exercícios

### Exercício 1
Compare os resultados obtidos com o conjunto de pesos `IMAGENET1K_V2`.

### Exercício 2
Compare os resultados obtidos com o modelo VGG16.